### Finding Prime Numbers

In [1]:
n = 500000
allnumbers = sc.parallelize(range(2, n), 8).cache()
composite = allnumbers.flatMap(lambda x: range(x*2, n, x)).repartition(8)# repartition(8)
prime = allnumbers.subtract(composite)
print (prime.take(10))

[17, 97, 113, 193, 241, 257, 337, 353, 401, 433]


In [2]:
# Find the number of elements in each parttion
def partitionsize(it): 
    s = 0
    for i in it:
        s += 1
    yield s

print (allnumbers.mapPartitions(partitionsize).collect())
print (composite.mapPartitions(partitionsize).collect())
print (prime.mapPartitions(partitionsize).collect())
print (prime.glom().collect()[1][0:4])

[62499, 62500, 62500, 62500, 62499, 62500, 62500, 62500]
[704805, 704790, 704800, 704800, 704800, 704799, 704800, 704816]
[0, 5169, 1, 5219, 0, 5206, 0, 5189, 0, 5165, 0, 5199, 0, 5191, 0, 5199]
[17, 97, 113, 193]


In [1]:
import math
def IsTrangleOrArea(x1,y1,x2,y2,x3,y3):
    return abs((x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)) / 2.0)
 
def IsInside(x1,y1,x2,y2,x3,y3,x,y):
 
    #三角形ABC的面积
    ABC = IsTrangleOrArea(x1,y1,x2,y2,x3,y3)
 
    # 三角形PBC的面积
    PBC = IsTrangleOrArea(x,y,x2,y2,x3,y3)
 
    # 三角形ABC的面积
    PAC = IsTrangleOrArea(x1,y1,x,y,x3,y3)
 
    # 三角形ABC的面积
    PAB = IsTrangleOrArea(x1,y1,x2,y2,x,y)
 
    return (ABC == PBC + PAC + PAB)

IsInside(10, 30, 20, 0, 10, 30, 10, 15)


False

In [4]:
numPartitions = 10
A = (0,0)
B = (10000,0)
C = (0,10000)

points = sc.textFile('points.txt',numPartitions)
pairs = points.map(lambda l: tuple(l.split()))
pairs = pairs.map(lambda pair: (int(pair[0]),int(pair[1])))
pairs.cache()

PythonRDD[5] at RDD at PythonRDD.scala:52

In [9]:


def f(iterator):
    exist = '1'
    pair = tuple((1,1))   
    for i in iterator:
        x, y = i
        if IsInside(A[0], A[1], B[0], B[1], C[0], C[1], x, y):
            yield exist
       
results = pairs.mapPartitions(f).collect()


In [10]:
count =0
for i in range(0, len(results)):
         count +=len(results[i])  
print(count)        
           

41


### Data Partitioning

In [3]:
data = [8, 96, 240, 400, 1, 800]
rdd = sc.parallelize(zip(data, data),4)
print (rdd.partitioner)
print  (rdd.glom().collect())
rdd = rdd.reduceByKey(lambda x,y: x+y)
print (rdd.glom().collect())
print (rdd.partitioner.partitionFunc)
rdd = rdd.sortByKey()
print (rdd.glom().collect())
print (rdd.partitioner.partitionFunc)

None
[[(8, 8)], [(96, 96), (240, 240)], [(400, 400)], [(1, 1), (800, 800)]]
[[(8, 8), (96, 96), (240, 240), (400, 400), (800, 800)], [(1, 1)], [], []]
<function portable_hash at 0x109144ae8>
[[(1, 1), (8, 8)], [(96, 96), (240, 240)], [(400, 400)], [(800, 800)]]
<function RDD.sortByKey.<locals>.rangePartitioner at 0x110e11158>


In [4]:
def partitionsize(it): yield len(list(it))
    
n = 40000

def f(x):
    return x / (n/8)

data1 = range(0, n, 16)
# data1 = range(0, n, 16) + range(0, n, 16)
data2 = range(0, n, 8)
rdd1 = sc.parallelize(zip(data1, data2), 8)
print (rdd1.mapPartitions(partitionsize).collect())
rdd2 = rdd1.reduceByKey(lambda x,y: x+y)
print (rdd2.mapPartitions(partitionsize).collect())
rdd3 = rdd1.partitionBy(8, f)
print (rdd3.mapPartitions(partitionsize).collect())
rdd4 = rdd1.reduceByKey(lambda x,y: x+y, partitionFunc=f)
print (rdd4.mapPartitions(partitionsize).collect())

[312, 312, 312, 312, 312, 312, 312, 316]
[2500, 0, 0, 0, 0, 0, 0, 0]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 28.0 failed 1 times, most recent failure: Lost task 3.0 in stage 28.0 (TID 132, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 140, in dump_stream
    for obj in iterator:
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/pyspark/rdd.py", line 1778, in add_shuffle_key
    yield pack_long(split)
  File "/usr/local/Cellar/a

In [ ]:
a = sc.parallelize(zip(range(10000), range(10000)), 8)
b = sc.parallelize(zip(range(10000), range(10000)), 8)
print a.partitioner
a = a.reduceByKey(lambda x,y: x+y)
b = b.reduceByKey(lambda x,y: x+y)
c = a.join(b)
print c.getNumPartitions()
print c.partitioner.partitionFunc
print c.glom().first()[0:4]

### Partitioning in DataFrames

In [10]:
data1 = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4]
data2 = [2, 2, 3, 4, 5, 3, 1, 1, 2, 3]
df = spark.createDataFrame(zip(data1, data2))
print df.rdd.getNumPartitions()
print df.rdd.glom().collect()

8
[[Row(_1=1, _2=2)], [Row(_1=1, _2=2)], [Row(_1=1, _2=3)], [Row(_1=2, _2=4), Row(_1=2, _2=5)], [Row(_1=2, _2=3)], [Row(_1=3, _2=1)], [Row(_1=3, _2=1)], [Row(_1=3, _2=2), Row(_1=4, _2=3)]]


In [11]:
df1 = df.repartition(6, df._1)
print df1.rdd.glom().collect()
df1.show()

[[], [], [Row(_1=2, _2=4), Row(_1=2, _2=5), Row(_1=2, _2=3), Row(_1=4, _2=3)], [Row(_1=3, _2=1), Row(_1=3, _2=1), Row(_1=3, _2=2)], [], [Row(_1=1, _2=2), Row(_1=1, _2=2), Row(_1=1, _2=3)]]
+---+---+
| _1| _2|
+---+---+
|  2|  4|
|  2|  5|
|  2|  3|
|  4|  3|
|  3|  1|
|  3|  1|
|  3|  2|
|  1|  2|
|  1|  2|
|  1|  3|
+---+---+



In [13]:
# A 'real' example from SF Express
# Prepare three relational tables

from pyspark.sql.functions import *

num_waybills = 1000
num_customers = 100

rdd = sc.parallelize((i, ) for i in xrange(num_waybills))
waybills = spark.createDataFrame(rdd).select(floor(rand()*num_waybills).alias('waybill'), 
                                             floor(rand()*num_customers).alias('customer')) \
                .repartition('waybill')\
                .cache()
waybills.show()
print waybills.count()

rdd = sc.parallelize((i, i) for i in xrange(num_customers))
customers = spark.createDataFrame(rdd, ['customer', 'phone']).cache()
customers.show()
print customers.count()

rdd = sc.parallelize((i, ) for i in xrange(num_waybills))
waybill_status = spark.createDataFrame(rdd).select(floor(rand()*num_waybills).alias('waybill'), 
                                                   floor(rand()*10).alias('version')) \
                      .groupBy('waybill').max('version').cache()
waybill_status.show()
print waybill_status.count()

+-------+--------+
|waybill|customer|
+-------+--------+
|     29|       1|
|     29|      35|
|     65|      42|
|    270|      25|
|    222|      61|
|    938|      15|
|    730|      66|
|    720|      98|
|    720|      80|
|    243|      53|
|    367|      67|
|     54|      88|
|    296|       0|
|    296|      51|
|     19|      13|
|    926|      93|
|    965|      72|
|    965|      32|
|     54|      63|
|     54|       9|
+-------+--------+
only showing top 20 rows

1000
+--------+-----+
|customer|phone|
+--------+-----+
|       0|    0|
|       1|    1|
|       2|    2|
|       3|    3|
|       4|    4|
|       5|    5|
|       6|    6|
|       7|    7|
|       8|    8|
|       9|    9|
|      10|   10|
|      11|   11|
|      12|   12|
|      13|   13|
|      14|   14|
|      15|   15|
|      16|   16|
|      17|   17|
|      18|   18|
|      19|   19|
+--------+-----+
only showing top 20 rows

100
+-------+------------+
|waybill|max(version)|
+-------+------------+
|     

In [3]:
# We want to join 3 tables together.
# Knowing how each table is partitioned helps optimize the join order.

waybills.join(customers, 'customer').join(waybill_status, 'waybill').show()
# waybills.join(waybill_status, 'waybill').join(customers, 'customer').show()

+-------+--------+-----+------------+
|waybill|customer|phone|max(version)|
+-------+--------+-----+------------+
|    474|      80|   80|           3|
|    964|      47|   47|           5|
|    191|      81|   81|           2|
|    222|      26|   26|           4|
|    293|      12|   12|           7|
|    442|      90|   90|           7|
|    278|      40|   40|           8|
|    705|      47|   47|           2|
|    243|      78|   78|           1|
|    926|      37|   37|           6|
|     54|      16|   16|           8|
|    847|      90|   90|           8|
|    348|      42|   42|           7|
|    502|      60|   60|           8|
|    113|      74|   74|           2|
|    155|      99|   99|           3|
|    385|      44|   44|           5|
|    241|      79|   79|           7|
|    414|      15|   15|           0|
|    196|       2|    2|           3|
+-------+--------+-----+------------+
only showing top 20 rows



### Threading

In [13]:
import threading
import random

partitions = 2
n = 500000 * partitions

# use different seeds in different threads and different partitions
# a bit ugly, since mapPartitionsWithIndex takes a function with only index
# and it as parameters
def f1(index, it):
    random.seed(index + 987231)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0

def f2(index, it):
    random.seed(index + 987232)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0

def f3(index, it):
    random.seed(index + 987233)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0
    
def f4(index, it):
    random.seed(index + 987234)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0
    
def f5(index, it):
    random.seed(index + 987245)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0

f = [f1, f2, f3, f4, f5]
    
# the function executed in each thread/job
def dojob(i):
    count = sc.parallelize(xrange(1, n + 1), partitions) \
              .mapPartitionsWithIndex(f[i]).reduce(lambda a,b: a+b)
    print "Worker", i, "reports: Pi is roughly", 4.0 * count / n

# create and execute the threads
threads = []
for i in range(5):
    t = threading.Thread(target=dojob, args=(i,))
    threads += [t]
    t.start()

# wait for all threads to complete
for t in threads:
    t.join()    
'''
for i in range(5):
    dojob(i)
'''

Worker 3 reports: Pi is roughly 3.141884
Worker 0 reports: Pi is roughly 3.141768Worker
 2 reports: Pi is roughly 3.142668
Worker 1 reports: Pi is roughly 3.142268
Worker 4 reports: Pi is roughly 3.143328



for i in range(5):
    dojob(i)
